In [1]:
# Install Gradio
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


For single model

In [7]:
# # Install Gradio
# !pip install gradio

# Import necessary libraries
import gradio as gr
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pickle  # Import pickle to load the saved tokenizer

# Load your pre-trained model
# Replace with your model path or loading logic
model = tf.keras.models.load_model('/content/imdb_sentiment_model_rnn.h5')

# Load the tokenizer from the saved file
with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

# Define preprocessing (using the loaded tokenizer)
def preprocess_text(input_text):
    # Tokenize and pad sequences using the loaded tokenizer
    sequence = tokenizer.texts_to_sequences([input_text])
    padded_sequence = pad_sequences(sequence, maxlen=200)  # Use your model's maxlen
    return padded_sequence

# Prediction function
def predict_sentiment(input_text):
    preprocessed_text = preprocess_text(input_text)
    prediction = model.predict(preprocessed_text)
    # Adjust thresholds or logic for positive/negative
    sentiment = "Positive" if prediction[0] > 0.5 else "Negative"
    return sentiment

# Create the Gradio interface
interface = gr.Interface(
    fn=predict_sentiment,
    inputs=gr.Textbox(lines=2, placeholder="Enter a review here..."),
    outputs="text",
    title="IMDB Sentiment Analysis",
    description="Enter an IMDB review to classify it as Positive or Negative."
)

# Launch the interface
interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://afb93e7b28f3ec865f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Combining all the models

In [6]:
# Import necessary libraries
import gradio as gr
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pickle  # Import pickle to load the saved tokenizer


# Load the models (RNN, LSTM, CNN)
rnn_model = tf.keras.models.load_model('/content/imdb_sentiment_model_rnn.h5')
lstm_model = tf.keras.models.load_model('/content/imdb_sentiment_model_lstm.h5')
cnn_model = tf.keras.models.load_model('/content/imdb_sentiment_model_cnn.h5')

# Load the tokenizer from the saved file
with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

# Define maxlen (Ensure this matches the padding length used during training)
MAXLEN = 200

# Define preprocessing (using the loaded tokenizer)
def preprocess_text(input_text):
    # Ensure input is not empty
    if not input_text.strip():
        return None  # Return None if input is empty

    # Tokenize and pad sequences using the loaded tokenizer
    sequence = tokenizer.texts_to_sequences([input_text])
    padded_sequence = pad_sequences(sequence, maxlen=MAXLEN, truncating='post', padding='post')  # Padding and truncation
    return padded_sequence

# Prediction function for all models
def predict_sentiment(input_text):
    # Preprocess the input text
    preprocessed_text = preprocess_text(input_text)

    # Check if input is empty or invalid
    if preprocessed_text is None:
        return "Please enter a valid review."

    # Get predictions from all three models
    rnn_prediction = rnn_model.predict(preprocessed_text)
    lstm_prediction = lstm_model.predict(preprocessed_text)
    cnn_prediction = cnn_model.predict(preprocessed_text)

    # Define a function to determine sentiment based on prediction
    def get_sentiment(prediction):
        return "Positive" if prediction[0] > 0.5 else "Negative"

    # Get sentiment for each model
    rnn_sentiment = get_sentiment(rnn_prediction)
    lstm_sentiment = get_sentiment(lstm_prediction)
    cnn_sentiment = get_sentiment(cnn_prediction)

    # Displaying example accuracies (this can be replaced with actual test set evaluation if available)
    result = (
        f"RNN Prediction: {rnn_sentiment}\n"
        f"LSTM Prediction: {lstm_sentiment}\n"
        f"CNN Prediction: {cnn_sentiment}\n"
    )



    return result

# Create the Gradio interface
interface = gr.Interface(
    fn=predict_sentiment,
    inputs=gr.Textbox(lines=2, placeholder="Enter a review here..."),
    outputs="text",
    title="IMDB Sentiment Analysis with Multiple Models",
    description="Enter an IMDB review to classify it as Positive or Negative using RNN, LSTM, and CNN models."
)

# Launch the interface
interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cd88d3771344eb6ac8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
